# Camera Calibration

In [1]:
import cv2 as cv
import os
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import math

In [2]:
#Image Inputs
images_dir = 'workspace\module_segmentation\Sample Set\Distortion Correction'
images_ext = '.JPG'
output_dir = 'workspace\module_segmentation\Sample Set\Distortion Corrected'

#Function settings
debug = True

In [3]:
#Load directory
images_list = os.listdir(images_dir)
images_list = [image_name for image_name in images_list if image_name.endswith(images_ext)]

#Define corner detection termination criteria
term_criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#Define chessboard internal corner dimensions (full 8x8 chess board has 7x7 internal corners)
corner_dim = (7,7)
corner_ct = corner_dim[0] * corner_dim[1]

#Define where corner points should be in real world
remap_pt = np.zeros((corner_ct,3), np.float32)
remap_pt[:,:2] = np.mgrid[0:corner_dim[0],0:corner_dim[1]].T.reshape(-1,2)

#Iterate through images and compile corners
remap_pts = []
corner_pts = []
it = 0
for image_file in images_list:
    
    #Load image 
    image_name = os.path.splitext(image_file)[0]
    image_path = images_dir + '/' + image_file
    image = cv.imread(image_path)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    #Initial corner detection
    ret, corners = cv.findChessboardCorners(gray, corner_dim, None)
    
    #Refine corner detection
    if ret:
        
        #Store remap points (constant)
        remap_pts.append(remap_pt)
        
        #Improve corner calculation and store
        corners_subpx = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), term_criteria)
        corner_pts.append(corners_subpx)
        
        #Debug display
        if debug:
            debug_image = cv.drawChessboardCorners(image, corner_dim, corners_subpx, ret)
            output_path = os.path.join(output_dir, image_name + '_corners' + images_ext)
            cv.imwrite(output_path, cv.cvtColor(debug_image, cv.COLOR_BGR2RGB))
        
        